# LangChain Cohere Playground ⛹

In [1]:
from google.colab import userdata
cohere_api_key = userdata.get('COHERE_API_KEY')

In [2]:
!pip install langchain cohere chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 883.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [19]:
from langchain.embeddings import CohereEmbeddings

In [20]:
embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key)

In [21]:
from langchain.chat_models import ChatCohere
chat = ChatCohere(cohere_api_key=cohere_api_key)

In [6]:
import csv

csv_file = open('/content/chunked_tafsir.csv')
csv_reader = csv.reader(csv_file)
csv_data = list(csv_reader)

parsed_data = [{
    "Surah": x[1],
    "Aya": x[2],
    "Tafsir": x[3],
} for x in csv_data]

In [7]:
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate

#Loading the tafsir into the vector database
docsearch = Chroma.from_texts(
    [x["Tafsir"] for x in parsed_data],
    embeddings,
    metadatas=parsed_data).as_retriever()



```
You are a helpful muslim at the masjid, providing very short wise words to fellow muslims.
When spoken to, your response should depend exclusively on tafsirs here: {context}
You speak at most one sentence.

Provide the proper islamic concepts based on the user's situation: {message}
```





```
For each page_content, finish the sentence: "You may be interested in this verse because it is about...."

Your objective is to very briefly explain WHY a user searching {message} may be interested in that page_content.

Put each page_content's sentence explanation into a python list.
If there are N page_contents then there MUST BE N explanations!
Ensure len(python_list)==N
Here are the Documents containing N page_contents: {context}

DO NOT RETURN ANYTHING OTHER THAN THE PYTHON LIST CONTAINING THE N STRING VALUES!!!  
```



In [116]:
r = docsearch.get_relevant_documents("Charity")
doc_dic = {}
documentsDocs = []
for docs in r:
    Document(
    page_content= docs.metadata['Tafsir'],
    metadata= {"source": (docs.metadata['Surah'], docs.metadata['Aya'])}
    )
  # print(docs.metadata['Surah'])
  # print()

    documentsDocs.append(Document)



In [151]:
documentsDocs

[langchain.schema.document.Document,
 langchain.schema.document.Document,
 langchain.schema.document.Document,
 langchain.schema.document.Document]

In [207]:
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatCohere

llm = ChatCohere(cohere_api_key=cohere_api_key)
chain_type = "map_reduce"

chain = load_summarize_chain(llm, chain_type=chain_type, verbose=False)

prompt = ChatPromptTemplate.from_template("""
A user searches "{message}" and the following result comes up: {context}

Your job is to summarize this result in relation to the user's search into ONE sentence.

Only respond with the sentence and nothing else.
""")


chain = prompt | chat
query = "I am interested in about the role of parents."
documents = docsearch.get_relevant_documents(query)


chain.batch([{"message": query, "context": documents[0]}, {"message": query, "context":documents[1]}, {"message": query, "context":documents[2]}])


[AIMessage(content='The role of parents is paramount as outlined by Islamic texts, which emphasize the importance of being kind and dutiful to parents, along with worshiping Allah alone and being charitable. Commands to treat parents well are mentioned in the Quran, and Prophet Muhammad emphasized the importance of this duty as well. \n\nWould you like me to expand on any of the points related to the role of parents as outlined by Islamic texts?'),
 AIMessage(content='According to the majority of scholars, Ibn Jarir, and Ath-Thawri, parents must consult with each other on all major decisions pertaining to their child rearing, especially in regards to weaning, as stated in Surah 2, Aya 233 of the Quran. \n\nWould you like to know more about the Quran?'),
 AIMessage(content="The Hadith warns against extravagance and miserliness when spending orphans' money but encourages spending it wisely. It also instructs the guardian of an orphan to return the orphan's property once they reach adulth

In [127]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import LLMChain
import requests

# Reference the quran verse(s): ({surah}:{aya})
prompt_template = """
A user googles "{message}" and the following N results comes up: {context}

Your job is to summarize EACH result in relation to the user's google search into ONE sentence.

Only summarize each result and do nothing else

"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=['context', 'message']
)

verse_rec_chain = (
    {
        "input_documents": lambda x: docsearch.get_relevant_documents(x["message"]),
        "message": lambda x: x["message"],
    }
    | load_qa_chain(chat, chain_type="stuff", prompt=PROMPT)
)

query = "I am interested in about the role of parents."
result = verse_rec_chain.invoke({"message": query})
print(result)

{'input_documents': [Document(page_content='And (remember) when We took a covenant from the Children of Israel, (saying): Worship none but Allah (alone) and be dutiful and good to parents, and to kindred, and to orphans and (the poor), and speak good to people and perform As-Salah and give Zakah Then you slid back, except a few of you, while you are backsliders. The Covenant that Allah took from the Children of Israel..Allah reminded the Children of Israel of the commandments that He gave them, and the covenants that He took from them to abide by those commands, and how they intentionally and knowingly turned away from all of that Allah commanded them to worship Him and to associate none with Him in worship, just as He has commanded all of His creatures, for this is why Allah created them Allah said, "And We did not send any Messenger before you (O Muhammad ." but We revealed to him (saying): La ilaha illa Ana none has the right to be worshipped but I (Allah), so worship Me (alone and 

In [128]:
print(result['output_text'])

Result 1: Allah commands us to worship him alone and to be dutiful and good to our parents, along with other kin, the orphans, the poor, and our neighbors. 

Result 2: Ibn Mas'ud said that the best deed is to pray on time, and the next best deed is to be kind to one's parents. Weaning a child after 2 years of age is permissible if both parents consent, otherwise it is forbidden. 

Result 3: Charity given to relatives is considered both sadaqah and silah (keeping relations). 

Result 4: Allah commands us to worship him alone and to be dutiful to our parents. Additionally, we are to treat orphans and the poor with kindness once we gain possession of their property. 

Result 5: Allah commands us to abstain from pride and boastfulness, to worship him alone, and to be kind to our parents, relatives, orphans, the poor, and our neighbors. 

Would you like help with anything else regarding the search results?


In [129]:
n_docs = 0
for r in result['input_documents']:
  n_docs+=1
n_docs

4

In [94]:
for r in result['input_documents']:
  print(r.metadata['Tafsir'])
  print()

anything more insane than deviating from this path and following the path of misguidance and deviation instead Is there more injustice than this Allah said, "Verily, joining others in worship with Allah is a great Zulm (wrong." indeed) (31:13)..Abu Al-`Aliyah and Qatadah said, "This Ayah (2:130) was revealed about the Jews who invented a practice that did not come from Allah and that defied the religion of Ibrahim Allahs statement, - "Ibrahim was neither a Jew nor a Christian, but he was a true Muslim Hanifa (to worship none but Allah alone." and he was not of Al-Mushrikin Verily, among mankind who have the best claim to Ibrahim are those who followed him, and this Prophet (Muhammad ) and those who have believed (Muslims) And Allah is the Wali (Protector and Helper) of the believers.) (3:67-68), testifies to this fact..Allah said next, "When his Lord said to him, "Submit (i e be a Muslim."! He said, "I have submitted myself (as a Muslim) to the Lord of the `Alamin (mankind, Jinn and al

# Retrieving MetaData

*Creating list of metadata values*

In [59]:
quran_data = {}
for page_content in result['input_documents']:
  for metadata in page_content:
    if (metadata[0] == "metadata"):
      for dictionary in metadata:
        if (type(dictionary)==dict):
          print((dictionary['Surah'], dictionary['Aya']))
          #Create Verse
          verse = (dictionary['Surah'], dictionary['Aya'])

          try:
            if dictionary['Tafsir'] not in quran_data[verse]:
              quran_data[verse].append(dictionary['Tafsir'])
          except:
            quran_data[verse] = dictionary['Tafsir']


print()
for keys in quran_data.keys():
  print(keys)

('2', '130,131,132')
('4', '123,124,125,126')
('3', '83,84,85')
('2', '120,121')

('2', '130,131,132')
('4', '123,124,125,126')
('3', '83,84,85')
('2', '120,121')


## Displaying Verses

In [10]:
# GET VERSE INFO with API
returned_verses = {}
i = 0
for verse_tuple in quran_data.keys():
  surah, ayas = verse_tuple
  aya_list = [int(x) for x in ayas.split(',')]
  eng_verse_chunk = ""
  arb_verse_chunk = ""
  tafsir_docs = quran_data[verse_tuple]
  for aya in aya_list:
    api_url = f"https://quranenc.com/api/v1/translation/aya/english_saheeh/{surah}/{aya}"
    response = requests.get(api_url)
    data = response.json()
    eng = data['result']['translation']
    arb = data['result']['arabic_text']
    # footnote = data['result']['footnotes']
    # print(arb)
    # print(eng)
    # print(f"({surah}:{aya})")
    eng_verse_chunk+=eng
    arb_verse_chunk+=arb + f"({aya})"
  returned_verses[i] = {"English": eng_verse_chunk,
                        "Arabic": arb_verse_chunk,
                        "Citation": f"({surah}:{ayas})",
                        "Docs": tafsir_docs}
  i+=1

In [11]:
returned_verses

{0: {'English': '(130) And who would be averse to the religion of Abraham except one who makes a fool of himself. And We had chosen him in this world, and indeed he, in the Hereafter, will be among the righteous.(131) When his Lord said to him, "Submit," he said, "I have submitted [in Islām][45] to the Lord of the worlds."(132) And Abraham instructed his sons [to do the same] and [so did] Jacob, [saying], "O my sons, indeed Allāh has chosen for you this religion, so do not die except while you are Muslims."',
  'Arabic': 'وَمَن يَرۡغَبُ عَن مِّلَّةِ إِبۡرَٰهِـۧمَ إِلَّا مَن سَفِهَ نَفۡسَهُۥۚ وَلَقَدِ ٱصۡطَفَيۡنَٰهُ فِي ٱلدُّنۡيَاۖ وَإِنَّهُۥ فِي ٱلۡأٓخِرَةِ لَمِنَ ٱلصَّٰلِحِينَ(130)إِذۡ قَالَ لَهُۥ رَبُّهُۥٓ أَسۡلِمۡۖ قَالَ أَسۡلَمۡتُ لِرَبِّ ٱلۡعَٰلَمِينَ(131)وَوَصَّىٰ بِهَآ إِبۡرَٰهِـۧمُ بَنِيهِ وَيَعۡقُوبُ يَٰبَنِيَّ إِنَّ ٱللَّهَ ٱصۡطَفَىٰ لَكُمُ ٱلدِّينَ فَلَا تَمُوتُنَّ إِلَّا وَأَنتُم مُّسۡلِمُونَ(132)',
  'Citation': '(2:130,131,132)',
  'Docs': 'anything more insane than devi

# Internet Search

In [12]:
from langchain.retrievers import CohereRagRetriever
from langchain.schema.document import Document

def _pretty_print(docs):
    for doc in docs:
        print(doc.metadata)
        print("\n\n" + doc.page_content)
        print("\n\n" + "-" * 30 + "\n\n")

rag = CohereRagRetriever(llm=chat)

In [14]:
#document search (with citations!)
docs = rag.get_relevant_documents(
    "What is 1+1?",
    source_documents=[
        Document(page_content="One plus One is 2!")
    ],
)
_pretty_print(docs)

{'id': 'doc-0', 'snippet': 'One plus One is 2!'}


One plus One is 2!


------------------------------


{'type': 'model_response', 'citations': [{'start': 8, 'end': 10, 'text': '2.', 'document_ids': ['doc-0']}], 'search_results': None, 'search_queries': None, 'token_count': {'prompt_tokens': 397, 'response_tokens': 27, 'total_tokens': 424, 'billed_tokens': 33}}


1 + 1 = 2. 

That is the correct sum of 1 + 1. Did you think it would equal another number?


------------------------------




In [239]:
#document search (with citations!)
docs = rag.get_relevant_documents(
    query = query
)
_pretty_print(docs)

{'id': 'web-search_9:0', 'snippet': 'Photo by Picsea on Unsplash\n\nWhat is the Role of a Parent\n\nWe try to answer the question What is the Role of a Parent? by taking into analysis the biological, psychological, social and financial dimensions. The parents’ roles are of the maximum importance for personality development of the children. Basic traditional roles of being a parent are: nurture and educate children, discipline them, manage home and financially support the family. Modern roles brought active participation of fathers in children care. The main idea is that parents must be child-centred and should aim to optimal growth and development of their children, to help them have satisfactory biological, social, psychological and emotional growth.', 'title': "What is the Role of a Parent. We try to answer the question What is… | by Big Phil's Nook | Medium", 'url': 'https://bigphill.medium.com/what-is-the-role-of-a-parent-acf892a600c'}


Photo by Picsea on Unsplash

What is the Rol

# Weaviate

In [4]:
!pip install weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 12.1 MB/s eta 0:00:00


In [10]:
import os
import weaviate
import json

auth_config = weaviate.auth.AuthApiKey(api_key='t25tkIKVncKsmW0ATSBqytZEjxC9c3VAjfSn')
client = weaviate.Client(
    url="https://cohere-hack-jrvynxdk.weaviate.network/",
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key":"UUxT220ExD2Zsrngo4olu2XuWjYsscpaVXDpnS0C"
    },
    startup_period=90
)
client.is_ready()

True

## Key-word Search

In [11]:
where_filter = {
  "path": ["tafsir"],
  "operator": "Like",
  "valueString": "prophet"
}

query_result = (
  client.query
  .get("quran", ["sura", "aya","tafsir"])
  .with_where(where_filter)
  .with_limit(3)
  .do()
)

print(json.dumps(query_result, indent=2))

{
  "data": {
    "Get": {
      "Quran": [
        {
          "aya": "1",
          "sura": 1,
          "tafsir": "Abu Hurayrah once performed the prayer and recited Bismillah aloud After he finished the prayer, he said, \"Among you, I perform the prayer that is the closest to the prayer of the Messenger of Allah Ad-Daraqutni, Al-Khatib and Al-Bayhaqi graded this Hadith Sahih Furthermore, in Sahih Al-Bukhari it is recorded that Anas bin Malik was asked about the recitation of the Prophet He said, \"His recitation was unhurried He then demonstrated that and recited, while lengthening the recitation of Bismillah Ar-Rahman Ar-Rahim, Also, in the Musnad of Imam Ahmad, the Sunan of Abu Dawud, the Sahih of Ibn Hibban and the Mustadrak of Al-Hakim - it is recorded that Umm Salamah said, \"The Messenger of Allah used to distinguish each Ayah during his recitation, - - - \"In the Name of Allah, the Most Gracious, the Most Merciful All praise and thanks be to Allah, the Lord of all that exist

In [220]:
query

'I am interested in about the role of parents.'

## Semantic Search

In [234]:
def semantic_search(query):
    nearText = {
        "concepts": [query],
#         "distance": -139.0,
    }

    properties = [
        "sura", "aya", "tafsir",
        "_additional {distance}"
    ]

    response = (
        client.query
        .get("quran", properties)
        .with_near_text(nearText)
        .with_limit(3)
        .do()
    )

    result = response['data']['Get']['Quran']

    return result


res = semantic_search(query)
res

[{'_additional': {'distance': -146.06438},
  'aya': '8,9',
  'sura': 29,
  'tafsir': 'And We have enjoined on man to be dutiful to his parents; but if they strive to make you associate with Me, which you have no knowledge of, then obey them not Unto Me is your return and I shall tell you what you used to do And for those who believe and do righteous deeds, surely, We shall admit them among the righteous...Allah commands His servants to be dutiful to parents, after urging them to adhere to belief in His Tawhid, because a persons parents are the cause of his existence So he must treat them with the utmost kindness and respect, his father for spending on him and his mother because of her compassion for him Allah says: - "And your Lord has decreed that you worship none but Him And that you be dutiful to your parents If one of them or both of them attain old age in your life, say not to them a word of disrespect, nor shout at them, but address them in terms of honor And lower unto them the 

## Hybrid

In [238]:
response = (
    client.query
    .get("Quran", ["tafsir", "sura", "aya"])
    .with_hybrid(
        query=query,
        alpha=0.75
    )
    .with_limit(3)
    .do()
)

print(json.dumps(response, indent=2))

{
  "data": {
    "Get": {
      "Quran": [
        {
          "aya": "15,16",
          "sura": 46,
          "tafsir": "And We have enjoined on man to be dutiful and kind to his parents His mother bears him with hardship And she delivers him with hardship And (the period of) his gestation and the weaning of him is thirty months, till when he attains full strength and reaches forty years, he says: \"My Lord! Grant me the power and ability that I may be grateful for Your favor which You have bestowed upon me and upon my parents, and that I may do righteous good deeds, such as please You, and make my offspring good Truly, I have turned to You in repentance, and truly, I am one of the Muslims They are those from whom We shall accept the best of their deeds and overlook their evil deeds (They shall be) among the dwellers of Paradise That is a promise of truth that they had been promised..Allahs Advice about the Parents.After mentioning that one must confess His Oneness, worship Him with 

# Weaviate Hybrid Search & Langchain Cohere

In [5]:
import os
import weaviate
import json

auth_config = weaviate.auth.AuthApiKey(api_key='t25tkIKVncKsmW0ATSBqytZEjxC9c3VAjfSn')
client = weaviate.Client(
    url="https://cohere-hack-jrvynxdk.weaviate.network/",
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key":"lwU3WyxaXvsIBvUIDcMUOMH80FzO316zgBdLRt7H"
    }
)
client.is_ready()

True

In [26]:
query = "The oneness of God. Feeling Grateful"

response = (
    client.query
    .get("Quran", ["tafsir", "sura", "aya"])
    .with_hybrid(
        query=query,
        alpha=0.75
    )
    .with_limit(3)
    .do()
)

In [27]:
documents = response['data']['Get']['Quran']

In [10]:
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatCohere

llm = ChatCohere(cohere_api_key=cohere_api_key)
chain_type = "map_reduce"

chain = load_summarize_chain(llm, chain_type=chain_type, verbose=False)

prompt = ChatPromptTemplate.from_template("""
A user searches "{message}" and the following result comes up: {context}

Your job is to summarize this result in relation to the user's search into ONE sentence.

Only respond with the sentence and nothing else. Do not ask any follow up questions!
""")


chain = prompt | llm

result = chain.batch([{"message": query, "context": documents[0]['tafsir']}, {"message": query, "context":documents[1]['tafsir']}, {"message": query, "context":documents[2]['tafsir']}])

In [11]:
import re

def remove_newlines_and_questions(text):
    # Remove everything after the first newline character
    text = re.sub(r"\n.*", "", text)
    text = re.sub(r'\.(.*?\?)', ".", text)
    text = re.sub(r"Muhammad", "Muhammad (saw)", text)

    # Remove any spaces at the beginning or end of the string
    text = text.strip()

    return text

In [24]:
summaries = []
for ai in result:
  cleaned = remove_newlines_and_questions(ai.content)
  summaries.append(cleaned)

for s in summaries:
  print(s)
  print()

Feeling grateful is an important part of recognizing the oneness of God; the Islamic belief that God is singular and the only true God. Allah, as described in the Quran, has provided mankind with many blessings, such as the night and day, and the ability to live and thrive on the earth. However, many deny Allah's grace and choose instead to worship idols.

The oneness of God is a central tenet of Islam, and those who do not believe in the Hereafter, their hearts are in denial and filled with pride at the idea of worshipping anyone but Allah, the One, the Unique, the Lone, and the Self-Sufficient. Allah knows the truth and will punish them for their arrogance.

Feeling grateful is part of the spiritual journey towards truly understanding the oneness of God, who is the one and only deity, with no partners or equals. This message can be found in several hadiths and verses in the Quran, such as Ayah 3 from Surah Al-Ilah in the Quran, which states "And your God is One God; there is only One

In [28]:
quran_data = []

quran_data.append((documents[0]['sura'], documents[0]['aya'], documents[0]['tafsir']))
quran_data.append((documents[1]['sura'], documents[1]['aya'], documents[1]['tafsir']))
quran_data.append((documents[2]['sura'], documents[2]['aya'], documents[2]['tafsir']))

In [14]:
quran_data

[(40,
  '61,62,63,64,65',
  'Allah, it is He Who has made the night for you that you may rest therein and the day for you to see Truly, Allah is full of bounty to mankind; yet, most of mankind give no thanks That is Allah, your Lord, the Creator of all things, La ilaha ila Huwa How then are you turning away Thus were turned away those who used to deny the Ayat of Allah Allah, it is He Who has made for you the earth as a dwelling place and the sky as a canopy, and has given you shape and made your shapes good and pure (looking) and has provided you with good things That is Allah, your Lord, so Blessed be Allah, the Lord of all that exists He is the Ever Living, La ilaha illa Huwa; so invoke Him making the religion for Him Alone All the praises and thanks be to Allah, the Lord of all that exits..Signs of the Power and Oneness of Allah..Allah reminds us of His grace towards His creation in that He has given them the night in which they rest and relax from their activities so that they can

In [16]:
import requests
# GET VERSE INFO with API
returned_verses = {}
i = 0
for verse_tuple in quran_data:
  surah, ayas, tafsir_docs = verse_tuple
  aya_list = [int(x) for x in ayas.split(',')]
  eng_verse_chunk = ""
  arb_verse_chunk = ""
  for aya in aya_list:
    api_url = f"https://quranenc.com/api/v1/translation/aya/english_saheeh/{surah}/{aya}"
    response = requests.get(api_url)
    data = response.json()
    eng = data['result']['translation']
    arb = data['result']['arabic_text']
    # footnote = data['result']['footnotes']
    # print(arb)
    # print(eng)
    # print(f"({surah}:{aya})")
    eng_verse_chunk+=eng
    arb_verse_chunk+=arb + f"({aya})"
  returned_verses[i] = {"English": eng_verse_chunk,
                        "Arabic": arb_verse_chunk,
                        "Citation": f"({surah}:{ayas})",
                        "Docs": tafsir_docs}
  i+=1

In [17]:
for k,v in returned_verses.items():
  print(v['English'])
  print(v['Arabic'])
  print(v['Citation'])
  print()

(61) It is Allāh who made for you the night that you may rest therein and the day giving sight.[1376] Indeed, Allāh is the possessor of bounty for the people, but most of them are not grateful.(62) That is Allāh, your Lord, Creator of all things; there is no deity except Him, so how are you deluded?(63) Thus were those [before you] deluded who were rejecting the signs of Allāh.(64) It is Allāh who made for you the earth a place of settlement and the sky a structure [i.e., ceiling] and formed you and perfected your forms and provided you with good things. That is Allāh, your Lord; then blessed is Allāh, Lord of the worlds.(65) He is the Ever-Living;[1377] there is no deity except Him, so call upon Him, [being] sincere to Him in religion. [All] praise is [due] to Allāh, Lord of the worlds.
ٱللَّهُ ٱلَّذِي جَعَلَ لَكُمُ ٱلَّيۡلَ لِتَسۡكُنُواْ فِيهِ وَٱلنَّهَارَ مُبۡصِرًاۚ إِنَّ ٱللَّهَ لَذُو فَضۡلٍ عَلَى ٱلنَّاسِ وَلَٰكِنَّ أَكۡثَرَ ٱلنَّاسِ لَا يَشۡكُرُونَ(61)ذَٰلِكُمُ ٱللَّهُ رَبُّكُمۡ 

## Pt.2: User Selects Verse
Let's say they selected the third verse

In [56]:
import cohere

generate_prompt = f"""
Tadabbur refers to the act of contemplation, reflection, or thoughtful consideration.
It is often associated with reflecting on the meanings of Quranic verses or contemplating the natural world as a means of deepening one's understanding and connection with God.
Tadabbur involves pondering the deeper meanings and implications of various aspects of life.

Using this definition, generate 3 deep and descriptive questions to assist one in doing tadabbur for the folloing text: {documents[2]['tafsir']}.

Ex for text regarding Monotheism:

1. Reflect upon the concept of monotheism and the idea that there is no deity but Allah. What does this mean in the context of our existence and the universe? How does this notion contrast with the idea of polytheism or the belief in many gods?
2. Consider the role of creation as a testimony to Allah's oneness. What does it mean that the heavens, the earth, and all creatures between them testify to Allah's uniqueness? How does the beauty and complexity of the natural world serve as a sign of Allah's existence and Oneness?
3. Contemplate the importance of the names "Ar-Rahman" and "Ar-Rahim" in this passage. What do these names imply about Allah's character and how do they relate to the concept of oneness? How do these names emphasize Allah's compassion and mercy towards creation?

"""

co = cohere.Client(cohere_api_key) # This is your trial API key
response = co.generate(
  model='command',
  prompt=generate_prompt,
  max_tokens=500,
  temperature=1.0,
  k=0,
  stop_sequences=[],
  return_likelihoods='NONE')


In [57]:
print(response.generations[0].text)
resp = response.generations[0].text

 Sure, here are three deep and descriptive questions to assist you in tadabbur for the provided text about monotheism:

1. contemplate the idea that Allah is the only deity worthy of worship. What does this mean in the context of our existence, where we encounter countless phenomena and distractions that might lead us to think that many forces are in play? How can we recognize and affirm the oneness of Allah in our daily lives and worship?

2. Consider the names "Ar-Rahman" and "Ar-Rahim" that appear in this passage. What do these names tell us about the nature of Allah and His relationship with creation? How do these names echo throughout the universe in different forms and experiences, testifying to Allah's eternal compassion and mercy?

3. Analyzing the concept of creation as a testimony to Allah's oneness. What is the significance of the heavens, the earth, and all creatures affirming Allah's uniqueness? How does the order, complexity, and beauty of the natural world serve as evide

In [58]:
p = remove_newlines_and_questions(resp)

print(p)

Sure, here are three deep and descriptive questions to assist you in tadabbur for the provided text about monotheism:


In [59]:
response.generations[0].text

' Sure, here are three deep and descriptive questions to assist you in tadabbur for the provided text about monotheism:\n\n1. contemplate the idea that Allah is the only deity worthy of worship. What does this mean in the context of our existence, where we encounter countless phenomena and distractions that might lead us to think that many forces are in play? How can we recognize and affirm the oneness of Allah in our daily lives and worship?\n\n2. Consider the names "Ar-Rahman" and "Ar-Rahim" that appear in this passage. What do these names tell us about the nature of Allah and His relationship with creation? How do these names echo throughout the universe in different forms and experiences, testifying to Allah\'s eternal compassion and mercy?\n\n3. Analyzing the concept of creation as a testimony to Allah\'s oneness. What is the significance of the heavens, the earth, and all creatures affirming Allah\'s uniqueness? How does the order, complexity, and beauty of the natural world serv